### Steps to Using PostgreSQL

1. Load PostgreSQL 
    ```bash
    module load data/PostgreSQL
    ```

2. Initialize the database
    ```bash 
    mkdir SQL_Data

    # Initialize the database cluster
    initdb -D SQL_Data

    # Start the database server
    pg_ctl -D SQL_Data -l logfile start
    ```

3. Activate the server
    - This example uses a server initialized on my system, to follow along, you will need to initialize your own server and place tables within the database. 
    ```bash
    # Evaluate the status of the server
    pg_ctl status -D SQL_Data

    # Alternative ways to start/stop the server
    pg_ctl -D SQL_Data stop
    pg_ctl -D SQL_Data start
    ```

4. Assign a user
    ```bash
    psql postgres  # Enter the PostgreSQL prompt

    CREATE DATABASE ng_database; # Create your database
    CREATE USER my_user WITH ENCRYPTED PASSWORD 'securepassword'; # Create the user within that database
    GRANT ALL PRIVILEGES ON DATABASE ng_database TO my_user; # Assign privileges to the user
    ```

5. Access the database
    ```bash
    psql -U my_user -h localhost -d ng_database
    ```

6. Create a table from a csv
    ```SQL
        CREATE TABLE al_residential (
        year INT,
        month INT,
        state TEXT,
        residential FLOAT,
        hdd20 FLOAT
    );
    \copy my_table(year, month, state, residential, hdd20) FROM 'Data/AL_Residential.csv' WITH (FORMAT csv, HEADER true);

    \q
    ```

In [1]:
import pandas as pd

df = pd.read_csv('Data/AL_Residential.csv')
df

,year,month,state,residential,HDD_20
0,1989,1,AL,238.903226,10.141640
1,1989,2,AL,251.571429,11.763194
2,1989,3,AL,238.451613,7.031389
3,1989,4,AL,157.400000,5.424619
4,1989,5,AL,92.129032,2.375010
...,...,...,...,...,...
403,2022,8,AL,20.838710,0.024494
404,2022,9,AL,21.900000,1.049530
405,2022,10,AL,38.516129,5.110879
406,2022,11,AL,98.533333,7.823489


In [3]:
!pip install psycopg2 SQLAlchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 25.9 MB/s eta 0:00:00


In [2]:
import psycopg2
import pandas as pd

# Database connection parameters
DB_NAME = "ng_database"
USER = "ehartley"
HOST = "localhost"  # Change if your database is hosted elsewhere
PORT = "5432"  # Default PostgreSQL port

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=USER,
    host=HOST,
    port=PORT
)
cur = conn.cursor()

# Define the table schema
create_table_query = """
CREATE TABLE IF NOT EXISTS nm_residential (
    year INT,
    month INT,
    state TEXT,
    residential FLOAT,
    hdd20 FLOAT
);
"""
cur.execute(create_table_query)
conn.commit()

# Load CSV using Pandas
csv_file = "Data/NM_Residential.csv"  # Adjust the path
df = pd.read_csv(csv_file)

# Insert data into PostgreSQL
for _, row in df.iterrows():
    cur.execute(
        "INSERT INTO nm_residential (year, month, state, residential, hdd20) VALUES (%s, %s, %s, %s, %s)",
        (row['year'], row['month'], row['state'], row['residential'], row['HDD_20'])
    )

# Commit and close
conn.commit()
cur.close()
conn.close()

print("Data successfully inserted into nm_residential.")

Data successfully inserted into nm_residential.


In [5]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Database credentials
host = "localhost"
database = "ng_database"
user = "ehartley"

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}@{host}/{database}')

# Read the load_zones table
df = pd.read_sql("SELECT * FROM nm_residential;", con=engine)

# Display first few rows
print(df.head())


   year  month state  residential      hdd20
0  1989      1    NM   181.387097  18.558140
1  1989      2    NM   167.714286  15.573355
2  1989      3    NM   107.903226  10.191154
3  1989      4    NM    65.333333   6.605651
4  1989      5    NM    35.838710   4.167347


In [3]:
import os
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

pghost = os.getenv("localhost", "")
pgdatabase = os.getenv("ng_database", "")
pguser = os.getenv("ehartley", "")

con = psycopg2.connect(database='ng_database', host='localhost', user='ehartley')
# use read-only session, because that's enough for this script and it's possible something
# weird could come through in the configuration info that gets passed to postgresql
con.set_session(readonly=True, autocommit=True)


In [4]:
# Query to list all tables in the public schema
query = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public';
"""

# Read tables into a DataFrame
df_tables = pd.read_sql(query, con)

# Display the list of tables
print(df_tables)

       table_name
0  al_residential
1  nm_residential


/tmp/ipykernel_3095351/1054658805.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql(query, con)


In [ ]:
query_nm = "SELECT * FROM nm_residential;"
# Read the periods table
df_nm = pd.read_sql(query_nm, con)
df_nm

/tmp/ipykernel_3095351/2454755599.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_lz = pd.read_sql(query_lz, con)


,year,month,state,residential,hdd20
0,1989,1,NM,181.387097,18.558140
1,1989,2,NM,167.714286,15.573355
2,1989,3,NM,107.903226,10.191154
3,1989,4,NM,65.333333,6.605651
4,1989,5,NM,35.838710,4.167347
...,...,...,...,...,...
403,2022,8,NM,30.870968,0.913051
404,2022,9,NM,27.000000,2.033279
405,2022,10,NM,54.161290,7.461242
406,2022,11,NM,167.933333,15.038542
